# Генерация текста с ruGPT-3

Сбер предоставляет доступ к следующим моделям:
- ruGPT3XL – 1.3B (*интерфейс с Hugging Face transformers не поддерживается*)
- ruGPT3Large – 760M
- ruGPT3Medium – 350M
- ruGPT3Small – 125M
- ruGPT2Large – 774M

Ссылки:  
- [Github](https://github.com/sberbank-ai/ru-gpts)
- [SberCloud](https://sbercloud.ru/ru/warp/gpt-3)
- [Хабр](https://habr.com/ru/company/sberbank/blog/524522/)
- [Hugging Face](https://huggingface.co/models?search=sberbank)
- [Демо](https://russiannlp.github.io/rugpt-demo/)

### Параметры

Выбор модели:

In [1]:
model = 'ruGPT3Large'  # ruGPT3Large
                       # ruGPT3Medium
                       # ruGPT3Small
                       # ruGPT2Large
            
if model == 'ruGPT3Large':
    model_name = 'sberbank-ai/rugpt3large_based_on_gpt2'
elif model == 'ruGPT3Medium':
    model_name = 'sberbank-ai/rugpt3medium_based_on_gpt2'
elif model == 'ruGPT3Small':
    model_name = 'sberbank-ai/rugpt3small_based_on_gpt2'
elif model == 'ruGPT2Large':
    model_name = 'sberbank-ai/rugpt2large'

Параметры генерации:

In [2]:
max_length = 100          # Длина генерируемого текста (включая "затравку")
num_return_sequences = 5  # Количество возвращаемых текстов

### Загрузка модели

In [3]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, set_seed
import numpy as np

Определяем GPU (если есть):

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

Инициализируем генератор случайных чисел:

In [5]:
set_seed(0)

Загружаем токенизатор и модель:

In [6]:
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

In [7]:
model = GPT2LMHeadModel.from_pretrained(model_name)

Присваиваем `pad_token_id` значение `eos_token_id`, иначе *HuggingFace* будет при каждой генерации предупреждать об этом ([ссылка](https://jaketae.github.io/study/gpt2/#setup)):

In [8]:
model.config.pad_token_id = model.config.eos_token_id

Переносим модель на GPU:

In [9]:
model.to(device);

Выводим количество параметров:

In [10]:
all_parameters = 0
for parameter in model.named_parameters():
    all_parameters += parameter[1].numel()

print(f'\nКоличество параметров: {all_parameters:,}')


Количество параметров: 760,300,032


In [11]:
print(f'Размер словаря: {tokenizer.vocab_size}')

Размер словаря: 50257


In [12]:
print(f'Максимальный размер входа: {model.config.max_position_embeddings}')

Максимальный размер входа: 2048


### "Затравка"

In [13]:
#text = 'Александр Сергеевич Пушкин родился в '
#text = 'Мой дядя самых честных правил '
#text = 'Аккредитация вуза - это '
#text = 'Смысл нашей жизни заключается в том, чтобы '
text = 'Смысл нашей жизни заключается в подготовке к аккредитации вуза, потому что '
#text = '2 + 2 = '
#text = '5 + 7 = '
#text = 'for i = 0 to n do '

---
Токенизация затравки:

In [14]:
input_ids = tokenizer.encode(text, return_tensors="pt")
input_ids = input_ids.to(device)
print(f'Длина input_ids[0] = {len(input_ids[0])}')
print(f'\ninput_ids[0]:\n{input_ids[0]}')

Длина input_ids[0] = 15

input_ids[0]:
tensor([  563,  6326,  2759,  1354,  9452,   282, 13148,   294, 46604,   553,
        39568,    16,  1548,   374,   225], device='cuda:0')


In [15]:
print(tokenizer.decode(input_ids[0]))

Смысл нашей жизни заключается в подготовке к аккредитации вуза, потому что 


### Жадный поиск (Greedy Search)

[Справка по функции `generate`](https://huggingface.co/transformers/main_classes/model.html#transformers.generation_utils.GenerationMixin.generate).

In [16]:
out = model.generate(
    input_ids=input_ids,           # Затравка
    max_length=max_length,         # Максимальная длина генерируемого текста (включая "затравку")
    return_dict_in_generate=True,  # Возвращать ли структуру ModelOutput или простой tuple
    output_scores=True             # Возвращать ли оценки для каждого токена
)

Выводим сгенерированный текст:

In [17]:
print(tokenizer.decode(out.sequences[0]))

Смысл нашей жизни заключается в подготовке к аккредитации вуза, потому что 
этот процесс является одним из самых важных и ответственных.

В этом году мы провели аккредитацию в соответствии с требованиями, которые предъявляются к аккредитации.

В этом году мы провели аккредитацию в соответствии с требованиями, которые предъявляются к аккредитации.

В этом году мы провели аккредитацию в соответствии с требованиями, которые предъявляются к аккредитации.

В этом году мы провели аккредитацию в


---
Выводим информацию об оценках:

In [18]:
print(f'Количество списков оценок: {len(out.scores)}')

Количество списков оценок: 85


In [19]:
i = 0
print(f'Список оценок для {i}-го элемента:\n\n{out.scores[0]}')

Список оценок для 0-го элемента:

tensor([[-5.0755,  8.9671, -5.4997,  ..., -4.6215, -2.8836, -1.6312]],
       device='cuda:0')


Находим токены с максимальными оценками для нескольких первых элементов:

In [20]:
n = 5
for i in range(n):
    id = np.argmax(out.scores[i][0].cpu())
    token = tokenizer.decode(id)
    print(f'id = {id},\ttoken = {token}')

id = 203,	token = 

id = 7944,	token = это
id = 266,	token = т
id = 4126,	token =  процесс
id = 1623,	token =  является


### Поиск по лучу (Beam Search)

In [21]:
num_beams = 5   # Количество лучей

Обычный поиск по лучу:

In [22]:
out = model.generate(
    input_ids=input_ids,
    max_length=max_length,
    num_beams=num_beams
)

print(tokenizer.decode(out[0]))

Смысл нашей жизни заключается в подготовке к аккредитации вуза, потому что 
у нас есть аккредитация, а у них нет аккредитации. 
У нас есть аккредитация, а у них нет аккредитации. 
У нас есть аккредитация, а у них нет аккредитации. 
У нас есть аккредитация, а у них нет аккредитации. 
У нас есть аккредитация, а у них нет аккредитации. 
У нас есть аккредитация, а у них


---
Запрещаем повтор N-грамм:

In [23]:
no_repeat_ngram_size = 2   # N-граммы, которые не должны повторяться

In [24]:
out = model.generate(
    input_ids=input_ids,
    max_length=max_length,
    num_beams=num_beams,
    no_repeat_ngram_size=no_repeat_ngram_size
)

print(tokenizer.decode(out[0]))

Смысл нашей жизни заключается в подготовке к аккредитации вуза, потому что 
у нас есть аккредитация, а у них ее нет.

- А как вы относитесь к тому, что некоторые вузы, в том числе и ваш, не могут пройти аккредитацию?
 - Я отношусь к этому с пониманием. У нас, как и в других вузах, есть определенные требования, которые предъявляются к вузам, и мы их выполняем. Но, к сожалению, у нас не


---
**Но**: если `no_repeat_ngram_size=2`, то в тексте не могут повторяться важные словосочетания из двух слов.

Выводим несколько вариантов текста (`num_return_sequences`):

In [25]:
def print_multiple_texts(output):
    for t in output:
        print(tokenizer.decode(t))
        print('\n*************')

In [26]:
out = model.generate(
    input_ids=input_ids,
    max_length=max_length,
    num_beams=num_beams,
    no_repeat_ngram_size=no_repeat_ngram_size,
    num_return_sequences=num_return_sequences
)

print_multiple_texts(out)

Смысл нашей жизни заключается в подготовке к аккредитации вуза, потому что 
у нас есть аккредитация, а у них ее нет.

- А как вы относитесь к тому, что некоторые вузы, в том числе и ваш, не могут пройти аккредитацию?
 - Я отношусь к этому с пониманием. У нас, как и в других вузах, есть определенные требования, которые предъявляются к вузам, и мы их выполняем. Но, к сожалению, у нас не

*************
Смысл нашей жизни заключается в подготовке к аккредитации вуза, потому что 
у нас есть аккредитация, а у них ее нет.

- А как вы относитесь к тому, что некоторые вузы, в том числе и ваш, не могут пройти аккредитацию?
 - Я отношусь к этому с пониманием. У нас, как и в других вузах, есть определенные требования, которые предъявляются к вузам, и мы их выполняем. Но, к сожалению, у нас нет

*************
Смысл нашей жизни заключается в подготовке к аккредитации вуза, потому что 
у нас есть аккредитация, а у них ее нет.

- А как вы относитесь к тому, что некоторые вузы, в том числе и ваш, не мог

### Сэмплирование с температурой

In [27]:
out = model.generate(
    input_ids=input_ids,
    do_sample=True,
    max_length=max_length,
    top_k=0,
    temperature=0.7,
    num_return_sequences=num_return_sequences
)

print_multiple_texts(out)

Смысл нашей жизни заключается в подготовке к аккредитации вуза, потому что 
каждый студент, который хочет учиться, должен быть аккредитован. 
Но мы всегда перед аккредитацией спрашиваем: хотите 
учиться? А если нет? А если у вас есть какие-то проблемы? 
А если у вас есть проблемы, то мы вас порекомендуем, потому что 
у нас есть список ваших проблем в этом году.

Суть аккредитации в том, что

*************
Смысл нашей жизни заключается в подготовке к аккредитации вуза, потому что 
это позволит более эффективно использовать ресурсы и получить большие
дополнительные возможности, чем если бы мы просто подали документы и
получили диплом.

- Антон, Вы не могли бы рассказать о том, как строятся отношения с высшими учебными заведениями 
и с промышленностью?

- Мы с ними работаем в тесном контакте. И я считаю, что одними из самых важных факторов
привле

*************
Смысл нашей жизни заключается в подготовке к аккредитации вуза, потому что 
нужно подтвердить свою квалификацию на уровне междуна

### Top-K Sampling

In [28]:
out = model.generate(
    input_ids,
    do_sample=True,
    max_length=max_length,
    top_k=50,
    num_return_sequences=num_return_sequences
)

print_multiple_texts(out)

Смысл нашей жизни заключается в подготовке к аккредитации вуза, потому что  это является условием для допуска к аккредитации, то есть это может быть аккредитация в других институтах.<s>
Ожидания, планы...... и т д.
Давно пора было это все осознать.
У меня был замечательный, очень удачный опыт построения отношений с мужчиной. И тут, неожиданно, мы попали на другого. И как так получилось - я этого не знаю. Но мы на него оба как

*************
Смысл нашей жизни заключается в подготовке к аккредитации вуза, потому что 
это одна из основных задач ректора, с этой точки зрения, 
у нас много делается, и надо эту задачу делать в том числе. Но у нас, как и во всем мире, сложилась система так называемого экзамена, или как его еще называют, устного экзамена. Что-то он будет проходить в Интернете, он состоится, и это событие войдет в историю. У нас, к сожалению, эта практика не работает.

*************
Смысл нашей жизни заключается в подготовке к аккредитации вуза, потому что », — сказал в беседе с

### Top-p (nucleus) семплирование

In [29]:
out = model.generate(
    input_ids,
    do_sample=True,
    max_length=max_length,
    top_p=0.92,
    top_k=0,
    num_return_sequences=num_return_sequences
)

print_multiple_texts(out)

Смысл нашей жизни заключается в подготовке к аккредитации вуза, потому что 
обеспечивает по направлению подготовки экономисты – экономисты.
Качество образовательных программ в разных вузах
Профессиональная программа повышения квалификации включает широкий круг дисциплин, в том числе
профессиональную модульную программу "Экономика организации"
(5-7 недель)
Специальность 15.04.04 "Экономика и управление на предприятии";

Особенности и требования к участникам программы повышения квалификации
Особ

*************
Смысл нашей жизни заключается в подготовке к аккредитации вуза, потому что 
т.н. "успешная экономика" и законодательство о сертификации качества высшего
образования создают благоприятные условия для коррупции и теневого бизнеса, - сказал
П.Троицкий.

Парламентарий считает, что "аттестационный процесс, при котором отдается предпочтение
частным вузам, в которых практикуется несправедливое распределение бюджетных средств,
наносящее ущерб качеству образования, является совершенно недоп

### Top-K & Top-p семплирование

Выводим несколько вариантов текста (`num_return_sequences`):

In [30]:
out = model.generate(
    input_ids,
    do_sample=True,
    max_length=max_length,
    top_p=0.92,
    top_k=50,
    num_return_sequences=num_return_sequences
)

print_multiple_texts(out)

Смысл нашей жизни заключается в подготовке к аккредитации вуза, потому что ­это наш ресурс для успешной защиты дипломной работы. Мы будем его использовать для улучшения условий работы, для повышения квалификации сотрудников в вузах, для формирования положительного имиджа. Это и называется — аккредитация. Я думаю, что сейчас наши вузы работают в этом направлении.
Если говорить о каких-то новшествах, то я бы хотела, чтобы наши вузы не уходили с рынка, чтобы мы получали конкурентные знания.

*************
Смысл нашей жизни заключается в подготовке к аккредитации вуза, потому что 
для прохождения аккредитации нам необходимо набрать определенный набор
коллективных экзаменов.

Набор таких экзаменов мы планируем с 2016 года
вместе с Министерством образования и науки РФ и 
при поддержке Министерства промышленности и торговли РФ.

Это касается не только документов вуза, но и документов его ректора.

Таким образом, по нашим планам, если сейчас все документы, что мы получаем, - это
бума

********